# Instacart Data Ingestion
# This notebook loads data from Azure Blob Storage into Databricks using a connection string.


In [0]:
# Define your storage account and container
storage_account_name = "instacartstorage123"
container_name = "instacartdata"
account_key = "RuSOWLQeMdQCJbWpL2Pl/p0gGDLsOX3BGApLf3kK/IQnYsSVA3TechIbOvOs/qBar+IpBxlsmS7e+AStKcq1hg=="

# Set the Spark configuration to access Azure Blob Storage
spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", account_key)


In [0]:
# Define the base path to access files via WASBS
base_path = f"wasbs://{container_name}@{storage_account_name}.blob.core.windows.net"


In [0]:
# Load datasets
products = spark.read.csv(f"{base_path}/products_cleaned.csv", header=True, inferSchema=True)
orders = spark.read.csv(f"{base_path}/orders_cleaned.csv", header=True, inferSchema=True)
order_products_train = spark.read.csv(f"{base_path}/order_products_train_cleaned.csv", header=True, inferSchema=True)
order_products_prior = spark.read.csv(f"{base_path}/order_products_prior_cleaned.csv", header=True, inferSchema=True)
aisles = spark.read.csv(f"{base_path}/aisles_cleaned.csv", header=True, inferSchema=True)
departments = spark.read.csv(f"{base_path}/departments_cleaned.csv", header=True, inferSchema=True)


In [0]:
# Save datasets as Delta Tables
products.write.format("delta").mode("overwrite").save("/delta/products")
orders.write.format("delta").mode("overwrite").save("/delta/orders")
order_products_train.write.format("delta").mode("overwrite").save("/delta/order_products_train")
order_products_prior.write.format("delta").mode("overwrite").save("/delta/order_products_prior")
aisles.write.format("delta").mode("overwrite").save("/delta/aisles")
departments.write.format("delta").mode("overwrite").save("/delta/departments")


In [0]:
# Create a database and register Delta tables
spark.sql("CREATE DATABASE IF NOT EXISTS instacart")

spark.sql("CREATE TABLE IF NOT EXISTS instacart.products USING DELTA LOCATION '/delta/products'")
spark.sql("CREATE TABLE IF NOT EXISTS instacart.orders USING DELTA LOCATION '/delta/orders'")
spark.sql("CREATE TABLE IF NOT EXISTS instacart.order_products_train USING DELTA LOCATION '/delta/order_products_train'")
spark.sql("CREATE TABLE IF NOT EXISTS instacart.order_products_prior USING DELTA LOCATION '/delta/order_products_prior'")
spark.sql("CREATE TABLE IF NOT EXISTS instacart.aisles USING DELTA LOCATION '/delta/aisles'")
spark.sql("CREATE TABLE IF NOT EXISTS instacart.departments USING DELTA LOCATION '/delta/departments'")


In [0]:
print("✅ Data ingestion complete!")
